# Savor Data

> Extract, transform, explore

Pipeline for retrieving data from the Airtable API and transforming it into something useful.

---

## Setup

In [1]:
# === Imports and config === #
from os import environ
from pprint import pprint
from pathlib import Path
from typing import List

from airtable import Airtable
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# Plotly imports
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio

# Set plotly default color theme to dark mode
pio.templates.default = "plotly_dark"

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

# Airtable authentication
load_dotenv(dotenv_path=".env")
base_key = environ.get("AIRTABLE_BASE_KEY")
api_key = environ.get("AIRTABLE_API_KEY")

---

## Pipeline

- [x] Extract + transform from "dimension" tables to match up related records' primary keys
- [x] Extract time-series data from `engage_log` into dataframe
- [ ] Transform / process the `engage_log` data to make ready for analysis

### Relations

In [2]:
def create_lookup(base_key: str, api_key: str, table_fields: dict) -> pd.Series:
    """Creates an ID lookup series from a set of Airtable tables."""
    series_index = []  # Index (id) array
    series_data = []  # Series data
    for table in tables_and_fields:  # Loop through tables to retrieve records and save in array
        airtable = Airtable(base_key, table, api_key=api_key)  # Connect to table
        records = airtable.get_all(fields=tables_and_fields[table])  # Retrieve records
        for record in records:  # Transform and save records to arrays
            series_index.append(record["id"])
            series_data.append(record["fields"]["name"])
    # Return series, indexed by ID
    return pd.Series(series_data, index=series_index)

In [3]:
# Related tables to match up via ID
tables_and_fields = {
    "mental": ["name"],
    "physical": ["name"],
    "tag": ["name"],
    "who": ["name"],
    "dose": ["name", "supp", "amt", "unit"],
    "location": ["name", "location", "city", "state"],
    "subloc": ["name"],
}

lookup = create_lookup(base_key, api_key, tables_and_fields)
# Confirm it worked by looking up name for id (in this ex, "Code")
lookup["recknqtgREfJulPie"]

'Code'

In [ ]:
# TODO: Create separate dataframes for related tables that hold additional info (dose, location)

### Time Series DataFrame

The data comes out of the API like this:

```py
[{'id': 'rec09tUYvKhC92ZSg',
  'fields': {'id_num': 24800,
   'session': ['recXt8s9a9fTRTbLf'],
   'name': '24800-Sta-Cod',
   'modified': '2021-04-29T22:05:23.000Z',
   'created': '2021-04-29T22:04:47.000Z',
   'project_location': ['recidY4IXWvLNWOp0'],
   'time_in': '2021-04-29T22:02:00.000Z',
   'mental': ['recknqtgREfJulPie'],
   'physical': ['recpSKLyNVgI0wdjf'],
   'tag': ['receKaJGCFL7i4a4g', 'recyPGKraDv5I0YQZ'],
   'subloc': ['rec0jakijbv2aJ2lj'],
   'mental_note': '# Savor Dash Data'},
  'createdTime': '2021-04-29T22:04:47.000Z'},
 {'id': 'recyGC5jmggOHxBWm',
  'fields': {'id_num': 24799,
   'session': ['recXt8s9a9fTRTbLf'],
   'name': '24799-Sta-Wor',
   'modified': '2021-04-29T21:58:15.000Z',
   'created': '2021-04-29T21:58:06.000Z',
   'project_location': ['recidY4IXWvLNWOp0'],
   'time_in': '2021-04-29T21:57:00.000Z',
   'mental': ['recV7J7xUq1TS4UaA'],
   'physical': ['recpSKLyNVgI0wdjf'],
   'subloc': ['rec0jakijbv2aJ2lj']},
  'createdTime': '2021-04-29T21:58:06.000Z'}]
```

And so needs to be transformed a little bit to be loaded into a DataFrame.

- Put `id` and all fields onto same level – i.e. de-nest the fields
- Convert the string IDs into their respective names

In [55]:
from datetime import datetime, timezone

def get_data_for_date_range(start: datetime, end: datetime, conn: Airtable, lookup: pd.Series, relations: List[str]) -> pd.DataFrame:
    """Retrieve Airtable data within the specified time range.
    Assumes that datetimes passed are in UTC, so convert to UTC beforehand.
    """
    # Loop through pages and records within pages, checking the time_in against the range
    records = []
    for page in conn.get_iter(sort=["-time_in"], page_size=50):
        # Page size of 50 = roughly # records in average day
        for record in page:
            # Airtable saves records in UTC
            time_in = datetime.fromisoformat(record["fields"]["time_in"][:-5] + "+00:00")
            # If inside range, append to list of raw data; else break from loop
            if time_in > end:
                continue
            elif time_in > start and time_in < end:
                # Extract id and fields into flat dictionary
                re_dict = record["fields"]
                re_dict["id"] = record["id"]

                # Remove columns not needed for current analysis
                for c in ["id_num", "name", "created", "modified"]:
                    del re_dict[c]

                # Convert timestamps into format usable by pandas (and into mountain time, for now)
                # TODO: convert to local time based on location.state
                re_dict["time_in"] = datetime.fromisoformat(re_dict["time_in"][:-5] + "-06:00")

                # Convert IDs into record names via lookup (created above)
                for rel in relations:  # Loop through list of fields relating to other tables
                    if rel in re_dict:  # Check if reference exists in record
                        for i, ref in enumerate(re_dict[rel]):  # Iter thru list of related records
                            # Replace each id in list with lookup match
                            re_dict[rel][i] = lookup[ref]

                # Append this record to the running list of records
                records.append(re_dict)
            else:
                break
        else:
            continue
        break  # Break outer loop when inner loop is broken

    # Load list of records into dataframe and return
    return pd.DataFrame.from_records(records)

In [6]:
# Create connection to engage_log table
engage_log = Airtable(base_key, "engage_log", api_key=api_key)

In [56]:
import dateutil

# Create start and end UTC datetimes
mdt = dateutil.tz.gettz("America/Denver")  # MDT timezone info
# Convert the MDT date to UTC
start = datetime(2021, 4, 26, tzinfo=mdt).astimezone(timezone.utc)
end = datetime.now(timezone.utc) # Now/today, in UTC

relations = [  # List of tables to convert via lookup
    "mental",
    "physical",
    "tag",
    "who",
    "subloc",
]

# Retrieve records for date range
df1_engage = get_data_for_date_range(start, end, engage_log, lookup, relations)

In [57]:
df1_engage.head()

,mental_note,session,subloc,tag,time_in,mental,physical,project_location,id,dose,physical_note,who,moment_log,location
0,# More Savor Building,[recgLfVD1QYzOBBCH],[Studio],"[savor, python, jupyter]",2021-05-02 03:05:00-06:00,[Code],[Stand],[recidY4IXWvLNWOp0],recYUJGCj6KjFf8SY,NaN,NaN,NaN,NaN,NaN
1,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,2021-05-02 03:00:00-06:00,[Think],[Cap],[recidY4IXWvLNWOp0],rec5wpcGPyH3HOa4u,[recqH7cYgoNWds9rc],NaN,NaN,NaN,NaN
2,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,2021-05-02 02:50:00-06:00,[Workflow],[Stand],[recidY4IXWvLNWOp0],rec1JgDA1NwUoSgU1,NaN,NaN,NaN,NaN,NaN
3,Watched some of Marcus' boring hotel tour vide...,[recgLfVD1QYzOBBCH],[Living Room],[youtube],2021-05-02 02:36:00-06:00,[Watch],[Snack],[recidY4IXWvLNWOp0],rec2H3q3ihfPrc44d,NaN,Ate a chocolate chip cookie with ice cream and...,[Marcus Ahlfeldt],NaN,NaN
4,NaN,[recgLfVD1QYzOBBCH],[Kitchen],NaN,2021-05-02 02:33:00-06:00,[Think],[Food prep],[recidY4IXWvLNWOp0],reci2c3QLq6mL2q1Y,NaN,"Decided to send it with a cookie, then added i...",NaN,NaN,NaN


### Calculating Durations

The first set of analyses I want to do with this data (for the dashboard) will revolve around the amount of time spent on various activities. Therefore, I'll need to calculate the duration of each one. I'm going to do it in this pipeline step so that the calculation doesn't have to run on the fly, which would likely slow things down a bit.

The pandas shift method will be used to create a new `time_out` column that will be the values of `time_in` shifted down one row – down because it's sorted by `time_in`, descending. With a `time_in` and `time_out` for each row, the duration can be calculated by taking the difference of the two.

In [65]:
# Copy; add time_out; calculate duration column
df2_engage = df1_engage.copy()
df2_engage["time_out"] = df2_engage["time_in"].shift(periods=1)
df2_engage["duration"] = df2_engage["time_out"] - df2_engage["time_in"]
df2_engage.head()

,mental_note,session,subloc,tag,time_in,mental,physical,project_location,id,dose,physical_note,who,moment_log,location,time_out,duration
0,# More Savor Building,[recgLfVD1QYzOBBCH],[Studio],"[savor, python, jupyter]",2021-05-02 03:05:00-06:00,[Code],[Stand],[recidY4IXWvLNWOp0],recYUJGCj6KjFf8SY,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,2021-05-02 03:00:00-06:00,[Think],[Cap],[recidY4IXWvLNWOp0],rec5wpcGPyH3HOa4u,[recqH7cYgoNWds9rc],NaN,NaN,NaN,NaN,2021-05-02 03:05:00-06:00,0 days 00:05:00
2,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,2021-05-02 02:50:00-06:00,[Workflow],[Stand],[recidY4IXWvLNWOp0],rec1JgDA1NwUoSgU1,NaN,NaN,NaN,NaN,NaN,2021-05-02 03:00:00-06:00,0 days 00:10:00
3,Watched some of Marcus' boring hotel tour vide...,[recgLfVD1QYzOBBCH],[Living Room],[youtube],2021-05-02 02:36:00-06:00,[Watch],[Snack],[recidY4IXWvLNWOp0],rec2H3q3ihfPrc44d,NaN,Ate a chocolate chip cookie with ice cream and...,[Marcus Ahlfeldt],NaN,NaN,2021-05-02 02:50:00-06:00,0 days 00:14:00
4,NaN,[recgLfVD1QYzOBBCH],[Kitchen],NaN,2021-05-02 02:33:00-06:00,[Think],[Food prep],[recidY4IXWvLNWOp0],reci2c3QLq6mL2q1Y,NaN,"Decided to send it with a cookie, then added i...",NaN,NaN,NaN,2021-05-02 02:36:00-06:00,0 days 00:03:00


In [59]:
df2_engage.dtypes

mental_note                            object
session                                object
subloc                                 object
tag                                    object
time_in             datetime64[ns, UTC-06:00]
mental                                 object
physical                               object
project_location                       object
id                                     object
dose                                   object
physical_note                          object
who                                    object
moment_log                             object
location                               object
time_out            datetime64[ns, UTC-06:00]
duration                      timedelta64[ns]
dtype: object

### DatetimeIndex

As the basic analysis of this data will be aggregating durations for various activities and time periods, I will index the data by `time_in`. This way, it's easy to group and aggregate using time periods directly – e.g. group by time-aware week: `df_engage.groupby(pd.Grouper(freq="W"))`.

In [67]:
# Set time_in as DatetimeIndex
# TODO: why is the drop=True not working?
df3_engage = df2_engage.set_index(pd.DatetimeIndex(df2_engage["time_in"])).drop(columns=["time_in"])
df3_engage.head()

,mental_note,session,subloc,tag,mental,physical,project_location,id,dose,physical_note,who,moment_log,location,time_out,duration
time_in,,,,,,,,,,,,,,,
2021-05-02 03:05:00-06:00,# More Savor Building,[recgLfVD1QYzOBBCH],[Studio],"[savor, python, jupyter]",[Code],[Stand],[recidY4IXWvLNWOp0],recYUJGCj6KjFf8SY,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2021-05-02 03:00:00-06:00,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,[Think],[Cap],[recidY4IXWvLNWOp0],rec5wpcGPyH3HOa4u,[recqH7cYgoNWds9rc],NaN,NaN,NaN,NaN,2021-05-02 03:05:00-06:00,0 days 00:05:00
2021-05-02 02:50:00-06:00,NaN,[recgLfVD1QYzOBBCH],[Studio],NaN,[Workflow],[Stand],[recidY4IXWvLNWOp0],rec1JgDA1NwUoSgU1,NaN,NaN,NaN,NaN,NaN,2021-05-02 03:00:00-06:00,0 days 00:10:00
2021-05-02 02:36:00-06:00,Watched some of Marcus' boring hotel tour vide...,[recgLfVD1QYzOBBCH],[Living Room],[youtube],[Watch],[Snack],[recidY4IXWvLNWOp0],rec2H3q3ihfPrc44d,NaN,Ate a chocolate chip cookie with ice cream and...,[Marcus Ahlfeldt],NaN,NaN,2021-05-02 02:50:00-06:00,0 days 00:14:00
2021-05-02 02:33:00-06:00,NaN,[recgLfVD1QYzOBBCH],[Kitchen],NaN,[Think],[Food prep],[recidY4IXWvLNWOp0],reci2c3QLq6mL2q1Y,NaN,"Decided to send it with a cookie, then added i...",NaN,NaN,NaN,2021-05-02 02:36:00-06:00,0 days 00:03:00


### Transformation (Expansion)

It's a little clunky to work with the relational fields in their current form: a list associated to a record. E.g. tags `[python, jupyter]`. Time to expand them all out into separate columns!

- [Expand arrays in cells](https://chrisalbon.com/python/data_wrangling/pandas_expand_cells_containing_lists/)
- [DataFrame JOINs](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join)

In [60]:
def expand_list_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """Expands arrays contained in columns and concatenates them
    back onto the original DataFrame.
    """
    df = df.copy()  # Make copy so original is not modified
    df_list = [None] * len(cols)  # Create list of same length
    for i, col in enumerate(cols):  # Iterate through column names
        # Expand column into its own dataframe
        df_list[i] = df[col].apply(pd.Series)
        # Rename and number columns
        if len(df_list[i].columns) > 1:
            df_list[i] = df_list[i].rename(columns = lambda x: f"{col}_{x}")
        else:  # If only one column, don't add suffix
            df_list[i] = df_list[i].rename(columns = lambda x: f"{col}")

        df = df.drop(columns=[col])  # Drop original column

    # Concatenate original with new dataframes
    return pd.concat([df] + df_list, axis=1)

In [68]:
# Expand all relation columns
relations = [
    "mental",
    "physical",
    "tag",
    "subloc",
    "moment_log",
    "who",
    "dose",
    "session",
    "project_location",
]

df4_engage = expand_list_cols(df3_engage, relations)
df4_engage.head()

,mental_note,id,physical_note,location,time_out,duration,mental_0,mental_1,physical_0,physical_1,tag_0,tag_1,tag_2,tag_3,subloc,moment_log,who_0,who_1,who_2,who_3,who_4,who_5,who_6,who_7,dose_0,dose_1,dose_2,dose_3,dose_4,session,project_location
time_in,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-02 03:05:00-06:00,# More Savor Building,recYUJGCj6KjFf8SY,NaN,NaN,NaT,NaT,Code,NaN,Stand,NaN,savor,python,jupyter,NaN,Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recgLfVD1QYzOBBCH,recidY4IXWvLNWOp0
2021-05-02 03:00:00-06:00,NaN,rec5wpcGPyH3HOa4u,NaN,NaN,2021-05-02 03:05:00-06:00,0 days 00:05:00,Think,NaN,Cap,NaN,NaN,NaN,NaN,NaN,Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recqH7cYgoNWds9rc,NaN,NaN,NaN,NaN,recgLfVD1QYzOBBCH,recidY4IXWvLNWOp0
2021-05-02 02:50:00-06:00,NaN,rec1JgDA1NwUoSgU1,NaN,NaN,2021-05-02 03:00:00-06:00,0 days 00:10:00,Workflow,NaN,Stand,NaN,NaN,NaN,NaN,NaN,Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recgLfVD1QYzOBBCH,recidY4IXWvLNWOp0
2021-05-02 02:36:00-06:00,Watched some of Marcus' boring hotel tour vide...,rec2H3q3ihfPrc44d,Ate a chocolate chip cookie with ice cream and...,NaN,2021-05-02 02:50:00-06:00,0 days 00:14:00,Watch,NaN,Snack,NaN,youtube,NaN,NaN,NaN,Living Room,NaN,Marcus Ahlfeldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recgLfVD1QYzOBBCH,recidY4IXWvLNWOp0
2021-05-02 02:33:00-06:00,NaN,reci2c3QLq6mL2q1Y,"Decided to send it with a cookie, then added i...",NaN,2021-05-02 02:36:00-06:00,0 days 00:03:00,Think,NaN,Food prep,NaN,NaN,NaN,NaN,NaN,Kitchen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recgLfVD1QYzOBBCH,recidY4IXWvLNWOp0


---

## Dashboard

Although this pipeline will ultimately be used in the dashboard app I'm building to explore my Savor data, I'll go through the process here of creating an example of the visualization that will be rendered in the app.

I calculated the aggregate durations in a somewhat clunky way in my first blog post [exploring Savor data](https://tobias.fyi/blog/savor-data-analysis-part-1#Monthly-totals). I think I can improve on it by skipping the intermediate step of "duration-hot encoding" the activity features, and aggregate them directly from the frame seen above.